In [ ]:
# Import the libraries

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure, io, img_as_ubyte
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu

In [ ]:
def gray_img(img):
    """
    Takes an image as the argument, returns it in grey scale.
    
    Parameters
    ----------
    img: The western blot image in the directory
    
    Returns
    ----------
    Same image in grey scale
    
    """
    
    if img.split(".")[-1] == "jpg":
        return img_as_ubyte(rgb2gray(io.imread(img)))
    
    else:
        return img_as_ubyte(rgb2gray(rgba2rgb(io.imread(img))))

In [ ]:
def thresh_img(grey_img):
    """
    Takes an grey scale image, returns threshold image with Otsu algorithm.
    
    Parameters
    ----------
    grey_img: the grey scale image of western blot provided by gray_img function
    
    Returns
    ----------
    Thresholded image from the grey scale image provided.
    
    """
    
    segmented = grey_img < threshold_otsu(grey_img)
    return segmented

In [ ]:
def band_labeling(original_img, threshold_img):
    """
    Takes an gray scale image, labels all the objects in it.
    
    Parameters
    ----------
    original_img: The raw form of western blot image provided by gray_img function
    threshold_img: Thresholded image provided by thresh_img function
    
    Returns
    ----------
    Thresholded image with all the objects labeled
    """
    
    return measure.label(threshold_img, connectivity = original_img.ndim)

In [ ]:
def band_separation(threshold_img):
    """
    Takes an thresholded image, finds the columns where each object is separated from the other.
    
    Parameters
    ----------
    thresh_img: Thresholded image with all the objects in it labeled
    
    Returns
    ----------
    A tuple of integers
    Every two inegters are for one object. The image is read from left to right. 
    The first integer is the first pixel of the object detected and the second one
    is the last pixel of the object detected.
    
    """
    
    col_img = np.any(threshold_img, axis = 0)
    separated_obj = np.flatnonzero(np.diff(col_img))
    return separated_obj

In [ ]:
def band_measurement(original_img, labeled_img, band_location, conc):
    """
    Calculates the are of each object in image from left to right,
    and saves it into a dictionary with corressponding does of the protein.
    
    Parameters
    ----------
    original_img: The raw western blot image provided from gray_img function
    labeled_img: Thresholded image with labeled object provided from band_labeling function
    band_location: The tuple of integers provided from band_separation function
    conc: List of strings, for all the concentrations used in the wb. Since objects are
    read from left to right, the list's order should be from left to right too.
    
    Returns
    ---------
    A dictionary whose keys are the concentrations and values are the are of the band.
    """
    measurements = {}
    start = 0
    end = 1
    
    for num in range(len(band_location) // 2):
        properties = measure.regionprops(labeled_img[:, band_location[start]: band_location[end]], 
                                         original_img[:, band_location[start]: band_location[end]])
        
        if len(properties) == 0:
            raise Exception("No object detected! Check the image.")
            
        elif len(properties) == 1:
            for ar in properties:
                measurements[conc[num]] = ar.area
                
        else:
            final = max([ar.area for ar in properties])
            measurements[conc[num]] = final
            
        start += 2
        end += 2
       
    return measurements

In [ ]:
def total_measurement(control, other):
    """
    Takes band measurements of an antibody and its control, divides each bands area to its corresponding control area
    
    Parameters
    ----------
    control: Dictionary from band_measurement function
    other: Dictionary from band_measurement function
    
    Returns
    ----------
    A dictionary whose keys are the concentration and values are other - control
    """
    
    return {pair[0]: int(pair[1]) / int(pair[2]) for pair in zip(other.keys(), other.values(), control.values())}

In [ ]:
def wb_graph(measurement, title, x_label, y_label, img_name):
    """
    Creates and saves graphs based on the info from band_measurement function
    
    Parameters
    ----------
    measurement: Dictionary from band_measurement function
    title: String, which will be the title of the grapgh
    img_name: String, which will be the name of the graph saved in the dorectory.
              The defualt format is png.
              
    Returns
    ---------
    A grapgh saved in the directory
    """
    plt.figure(figsize = (10, 7))
    plt.bar(measurement.keys(), measurement.values())             
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title) 
    plt.savefig(img_name)

In [ ]:
def multiple_blots(pathway_name, concs):
    """
    Calculates the bands for multiple blots from multiple images.
    
    Paramters
    ---------
    pathway_name: Name of the pathway (the antibody) used. It is assumed that
    each image name after the pathway looked into. Giving example, if working with
    Phospho-VASP, then the image should be named after it like pvasp.format. For 
    multiple images, pvasp1.format, pvasp2.format and so on is desired.
    concs: List of concentrations of the objects in the blot from left to right.
    
    Returns
    ---------
    A dictionary whose keys are the concentrations, from concs list, and values are a list from the measured 
    objects from the blots. 
    """
    total_results = []
    mid_result = []
    
    for item in os.listdir():
        if pathway_name.lower() in item.lower():
            
            result = band_measurement(gray_img(item), band_labeling(gray_img(item), thresh_img(gray_img(item))), 
                             band_separation(thresh_img(gray_img(item))), concs)
            
            total_results.append(result)
            
        else:
            continue
            
    
    for item in total_results:
        if len(mid_result) == 0:
            mid_result.append([*item.keys()])
            mid_result.append([*item.values()])

        else:
            mid_result.append([*item.values()])
        
    final = {x[0]: np.array([*x[1:]]) for x in list(zip(*mid_result))}
    return final

In [ ]:
def multiple_bars(mult_measures, title, file_name):
    """
    Creates a multiple bar grapgh from a dictionary
    
    Parameters
    ----------
    mult_measures: A dictionary provided by multiple_blots function.
    title: str, will be used as the title of the bar graph.
    file_name: str, will be used to save the bar in the directory. 
    """
    
    final = [[x[y] for x in [*mult_measures.values()]] for y in range(len([*mult_measures.values()][0]))]
    w = 0.3
    bar0 = np.arange(len(final[0]))
    bar1 = np.arange(len(final[0]))
    
    for items in range(len(final)):
    
        plt.bar(bar1, final[items], w, label = "Set " + str(items),
               yerr = np.std(final[items]), ecolor = 'black', capsize = 5)
        bar1 = [i + w for i in bar1]


    plt.xlabel("Concentrations")
    plt.ylabel("Quantifications")
    plt.xticks(bar0 + w / 2, test.keys())
    plt.legend(loc = "upper right", bbox_to_anchor = (1.21, 1.02))
    plt.title(title)
    plt.savefig(file_name)